<a href="https://colab.research.google.com/github/jhaashwaniii/NLP/blob/main/ASS_2a(HMM_WSD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
kfold = KFold(5, shuffle=True, random_state=1)

In [ ]:
import nltk
nltk.download('semcor')
! unzip -o /root/nltk_data/corpora/semcor.zip -d /root/nltk_data/corpora

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
semcor.tagged_sents(tag='sem')
data = semcor.tagged_sents(tag='sem')

In [ ]:
sentence_with_tags = []
for sent in data:
  tag_sent = []
  for word in sent:
    if (type(word)==nltk.tree.Tree):
      tag_sent.append(word)
  sentence_with_tags.append(tag_sent)
  sent_lemma = []
for sent in sentence_with_tags:
  temp = []
  for tree in sent:
    temp.append(tree.label())
  sent_lemma.append(temp)

In [13]:
syn_id = []
words = []
for sent in sent_lemma:
  temp_syn = []
  temp_word = []
  for word in sent:
    if(type(word)==nltk.corpus.reader.wordnet.Lemma):
      word =str(word)
      word = word.replace("Lemma('",'').replace("')",'')
    temp = word.split(".")
    res = ".".join(temp[:3]), ".".join(temp[3:])
    if(len(wn.synsets(res[1]))!=0):
      temp_syn.append(res[0])
      temp_word.append(res[1])
  words.append(temp_word)
  syn_id.append(temp_syn)
    


In [10]:
nltk_data = []
for i in range(len(words)):
  nltk_data.append(list(zip(words[i],syn_id[i])))
  

In [14]:
train_data_arr=[]
test_data_arr=[]
for train, test in kfold.split(nltk_data):
    train_set_temp=[]
    test_set_temp=[]
    for i in range(len(train)):
        index=train[i]
        train_set_temp.append(nltk_data[index])
    train_data_arr.append(train_set_temp)
    for i in range(len(test)):
        index=test[i]
        test_set_temp.append(nltk_data[index])
    test_data_arr.append(test_set_temp) 

In [15]:
train_set=train_data_arr[0]
test_set=test_data_arr[0]

In [16]:
for sent in range(len(train_set)):
  train_set[sent].insert(0,('$$', '$$'))

In [18]:
#Setting tag counter to zero for all syn_id
#syn_id_count is a dictionary with key= syn_id (example seed.n.02 and value is count in corpus) 
syn_id_set={tag for block in train_set for word,tag in block}
syn_id_count={}
for each_id in syn_id_set:
    syn_id_count[each_id]=0

In [19]:
#Getting count of each syn_id in the training corpus
for block in train_set:
    for word, tag in block:
        syn_id_count[tag]+=1

In [20]:
#Creating count of Unique word,syn_id combinations
word_tag_count={} #This is adictionary whoe key is a tuple (word, syn_id) and value is the count of word, syn_id in the corpus
for block in train_set:
    for word_tag_combo in block:
        try:
            word_tag_count[word_tag_combo]+=1
        except:
            word_tag_count[word_tag_combo]=1

In [21]:
#Creating bigrams from training set
bigram_blocks=[]
for s in train_set:
    bi=list(nltk.bigrams(s))
    bigram_blocks.append(bi)

In [22]:
#This is a dictionaty whose key is a tuple (state1_tag,state2_tag) and value is count in the corpus
transitional_state_count={}

In [23]:
for block in bigram_blocks:
    for bigram in block:
        state1=bigram[0][1]
        state2=bigram[1][1]
        transition_tuple=(state1,state2)
        try:
            transitional_state_count[transition_tuple]+=1
        except:
            transitional_state_count[transition_tuple]=1

In [24]:
transitional_state_count[('manner.n.01','election.n.01')]

1

In [25]:
#This is a dictionaty whose key is a tuple (state1_tag,state2_tag) and value is probabllity in the corpus
tag_transi_p={}

In [26]:
#Calculation of transitional probablity
for transition in transitional_state_count.keys():
    state1=transition[0]
    tag_transi_p[transition]=transitional_state_count[transition]/syn_id_count[state1]



In [27]:
tag_transi_p[('manner.n.01','election.n.01')]

0.005076142131979695

In [28]:
#Unique pos tags using set {} data structure in Python
unique_words ={word for block in train_set for word,tag in block}
print(len(unique_words))

18610


In [29]:
#finding transition prob from prob matrix
def find_transition(x):
  try:
    return tag_transi_p[x]
  except:
    return 0


In [30]:
find_transition(('manner.n.01','election.n.01'))

0.005076142131979695

In [31]:
#viterbi algo
def viterbi_algo(sent):
    tag_seq_predicted = [] 
    path_info = [] #list of dictionaries for each state_info          path_info[step][tag] = (prob, prev_tag)
    for step,word in enumerate(sent):  #step will be index and word will be corresponding element
    
      
      step_info = {} #dictionary of tag:(prob, prev state) for each step 
      syn_tags = wn.synsets(word)
      syn_tags = [str(w).replace("Synset('",'').replace("')",'') for w in syn_tags]
      if step == 0:
        for tag in syn_tags:
          try:
            step_info[tag] = (find_transition(('$$',tag))*word_tag_emi_p[(word,tag)],'$$') #since we are not storing emi prob for each word tag combination 
          except:                                                                      #if it is not there(words not in train set : assign emi prob:0.0000000001 )
            step_info[tag] = (find_transition(('$$',tag))*0.0000000001,'$$')
          
          
      else:
        prev_step_info = path_info[step-1] 
        
        for tag in syn_tags:
        
          temp = {} #{ prob from prev_tag : prev_tag } 
          for prev_tag in prev_step_info.keys(): #finding paths from all prev tags to current tag and selecting max among them
            try:
              node_prob  = prev_step_info[prev_tag][0]*find_transition((prev_tag,tag))*word_tag_emi_p[(word,tag)]
            except:
              node_prob  = prev_step_info[prev_tag][0]*find_transition((prev_tag,tag))*0.0000000001
            temp[node_prob] = prev_tag
          
          max_node_prob = max(temp.keys())
          step_info[tag] = (max_node_prob, temp[max_node_prob])
            
           
      path_info.append(step_info)
      
    last = path_info[len(sent)-1]  
    li = [last[l] for l in last ]   
    res = max(li, key = lambda i : i[0]) 
    for l in last.keys(): 
      if(last[l]==res):
        final_tag = l
    tag_seq_predicted.append(final_tag) 
    
    t = len(sent)-1 
    if(t>0):              
      tag_seq_predicted.append(res[1])   
      for i in range(1,t):         
      
       res = path_info[t-i][res[1]]
       tag_seq_predicted.append(res[1])
    tag_seq_predicted.reverse()    
    return(tag_seq_predicted)
     

In [32]:
#spliting test set to test_set_words and test_set_tags
test_set_words = []
test_set_tags = []
i=0
for sentence in test_set:
  if(len(sentence)>0):
    words, tags = map(list,zip(*sentence))
    test_set_words.append(words)
    test_set_tags.append(tags)

In [33]:

correct = 0
wrong = 0
test_set_correct_tag_counter={}
test_set_true_tag_counter={}
test_set_predicted_tag_counter={}
confusion_matrix={}
for i in range(len(test_set_words)):
   
    tag_seq_predicted = viterbi_algo(test_set_words[i])
    for j in range(len(tag_seq_predicted)):
        true_tag=test_set_tags[i][j]
        predicted_tag=tag_seq_predicted[j]
        try:
            confusion_matrix[(true_tag,predicted_tag)]+=1
        except:
            confusion_matrix[(true_tag,predicted_tag)]=1
        try:
            #Counting total count provided by dataset for each tag category 
            test_set_true_tag_counter[true_tag]+=1
        except:
            test_set_true_tag_counter[true_tag]=1
        try:
            #Counting total count provided by Viterbi Algo for each tag category 
            test_set_predicted_tag_counter[predicted_tag]+=1
        except:
            test_set_predicted_tag_counter[predicted_tag]=1
            
        if (tag_seq_predicted[j]==test_set_tags[i][j]):
            correct_tag=tag_seq_predicted[j]
            correct +=1
            try:
                #Counting cases where model predicted corrently for each tag category
                test_set_correct_tag_counter[correct_tag]+=1
            except:
                test_set_correct_tag_counter[correct_tag]=1
        else:
            wrong+=1
print(correct, wrong)
print('accuracy',correct/ (correct+ wrong))

9747 35284
accuracy 0.21645088938731097


In [34]:
print(test_set_correct_tag_counter)
print(test_set_true_tag_counter)
print(test_set_predicted_tag_counter)

{'september.n.01': 6, 'october.n.01': 4, 'investigate.v.02': 4, 'potential.a.01': 6, 'hard-fought.s.01': 1, 'voter.n.01': 2, 'experienced.a.01': 4, 'legislature.n.01': 1, 'enabling.a.01': 1, 'in_fact.r.01': 8, 'effectively.r.02': 3, 'elective.a.01': 1, 'unmeritorious.s.01': 1, 'four.s.01': 31, 'hire.v.01': 4, 'intern.n.01': 1, 'extern.n.01': 1, 'location.n.01': 3, 'legislator.n.01': 2, 'enabling_legislation.n.01': 1, 'equitable.a.01': 3, 'employee.n.01': 16, 'not.r.01': 350, 'marriage.n.02': 1, 'more.r.02': 3, 'mayor.n.01': 2, 'political.a.03': 3, 'city_council.n.01': 1, 'tenure.n.01': 1, 'january.n.01': 5, "governor's_race.n.01": 1, 'wednesday.n.01': 2, 'about.r.07': 26, 'thirty.s.01': 13, 'million.s.01': 10, 'highway.n.01': 5, 'test_case.n.01': 1, 'then.r.01': 17, 'sunday.n.01': 1, 'night.n.01': 1, 'revoke.v.02': 3, 'increased.a.01': 8, 'spend.v.02': 2, 'reportedly.r.01': 2, 'call.n.01': 1, 'ordinary.n.01': 1, 'person.n.01': 217, 'besides.r.02': 106, 'soon.r.01': 15, 'very.r.01': 4, 

In [35]:
confusion_matrix_perct={}
for tup in confusion_matrix.keys():
    true_tag=tup[0]
    confusion_matrix_perct[tup]=round(100*(confusion_matrix[tup]/test_set_true_tag_counter[true_tag]),1)

In [36]:
print(test_set_correct_tag_counter)
print(test_set_true_tag_counter)
print(test_set_predicted_tag_counter)

{'september.n.01': 6, 'october.n.01': 4, 'investigate.v.02': 4, 'potential.a.01': 6, 'hard-fought.s.01': 1, 'voter.n.01': 2, 'experienced.a.01': 4, 'legislature.n.01': 1, 'enabling.a.01': 1, 'in_fact.r.01': 8, 'effectively.r.02': 3, 'elective.a.01': 1, 'unmeritorious.s.01': 1, 'four.s.01': 31, 'hire.v.01': 4, 'intern.n.01': 1, 'extern.n.01': 1, 'location.n.01': 3, 'legislator.n.01': 2, 'enabling_legislation.n.01': 1, 'equitable.a.01': 3, 'employee.n.01': 16, 'not.r.01': 350, 'marriage.n.02': 1, 'more.r.02': 3, 'mayor.n.01': 2, 'political.a.03': 3, 'city_council.n.01': 1, 'tenure.n.01': 1, 'january.n.01': 5, "governor's_race.n.01": 1, 'wednesday.n.01': 2, 'about.r.07': 26, 'thirty.s.01': 13, 'million.s.01': 10, 'highway.n.01': 5, 'test_case.n.01': 1, 'then.r.01': 17, 'sunday.n.01': 1, 'night.n.01': 1, 'revoke.v.02': 3, 'increased.a.01': 8, 'spend.v.02': 2, 'reportedly.r.01': 2, 'call.n.01': 1, 'ordinary.n.01': 1, 'person.n.01': 217, 'besides.r.02': 106, 'soon.r.01': 15, 'very.r.01': 4, 

In [ ]:
print(sum(test_set_true_tag_counter.values()))
print(sum(test_set_predicted_tag_counter.values()))
print(sum(test_set_correct_tag_counter.values()))

In [38]:
#Unique pos tags using set {} data structure in Python
test_tag_set={tag for block in test_set for word,tag in block}
print(len(test_tag_set))

#Setting tag counter to zero for all syn_id
#syn_id_test_count is a dictionary with key= syn_id (example seed.n.02 and value is count in corpus) 
syn_id_test_count={}
for each_id in test_tag_set:
    syn_id_test_count[each_id]=0
    
#Getting count of each syn_id in the test corpus
for block in test_set:
    for word, tag in block:
        syn_id_test_count[tag]+=1    

12204


In [39]:
#weight_each_test_tag stores % population of each tag in test dataset
weight_each_test_tag={}
total=sum(syn_id_test_count.values())
for tag in syn_id_test_count.keys():
    weight_each_test_tag[tag]=syn_id_test_count[tag]/total
    

In [40]:
#Calculating precision, recall and F1 Score at Tag Level
precision_each_tag={}
recall_each_tag={}
f1score_each_tag={}
for tag in test_tag_set:
    try:
        precision_each_tag[tag]=test_set_correct_tag_counter[tag]/test_set_predicted_tag_counter[tag]
        precision=precision_each_tag[tag]
    except:
        precision_each_tag[tag]='NA'
        
    try:
        recall_each_tag[tag]=test_set_correct_tag_counter[tag]/test_set_true_tag_counter[tag]
        recall=recall_each_tag[tag]
    except:
        recall_each_tag[tag]='NA'
        
    
    try:
        f1score_each_tag[tag]=(2*precision_each_tag[tag]*recall_each_tag[tag])/(precision_each_tag[tag]+recall_each_tag[tag])
    except:
        f1score_each_tag[tag]='NA'

In [43]:
precision=0
recall=0
f1score=0
for tag in test_tag_set:
    if precision_each_tag[tag]!='NA':
        precision+=precision_each_tag[tag]*weight_each_test_tag[tag]
    if recall_each_tag[tag]!='NA':
        recall+=recall_each_tag[tag]*weight_each_test_tag[tag]
    if f1score_each_tag[tag]!='NA':
        f1score+=f1score_each_tag[tag]*weight_each_test_tag[tag]    

In [44]:
print(precision)
print(recall)
print(f1score)

0.3943757366102401
0.21645088938732315
0.2353852086173685
